<h1 align=center><font size = 5> Toranto Neighbourhood Research </font></h1>



## Introduction
This case is from IBM Data Science professional - Applied Data Science Capstone; 

## Table of Contents

The notebook will contain the following sections:

1. Foursquare API deployment
2. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

import requests
from bs4 import BeautifulSoup

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Get the page info to Pandas DataFrame

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup_html = BeautifulSoup(source, 'lxml')
#print(soup_html.prettify())

In [4]:
table_string = soup_html.find_all('table', 'wikitable sortable')[0]

In [5]:
pd_table = pd.read_html(str(table_string))[0]
pd_table.columns = pd_table.iloc[0]
pd_table = pd_table[1:]

### Delete Borough with "Not assigned." 

In [6]:

borough_assigned = pd_table[pd_table['Borough']!='Not assigned'].reset_index(drop=True)

###  Assign neighourhood with "Not assigned." 

In [7]:
borough_assigned[borough_assigned['Neighbourhood']=='Not assigned'] = borough_assigned['Borough']

In [8]:
#Check if still exist 'Not assigned'
sum(borough_assigned['Neighbourhood']=='Not assigned')

0

### Combine more Neighbourhoods for same postal code area

In [9]:

def join_array(arr):
    return ",".join(arr)

groupby_Postcode = borough_assigned.groupby('Postcode').agg({'Neighbourhood': join_array})
neighbourhoods = pd.merge(groupby_Postcode, borough_assigned, on='Postcode')
neighbourhoods = neighbourhoods.drop('Neighbourhood_y', axis=1)
neighbourhoods.columns=['Postcode', 'Neighbourhood', 'Borough']
neighbourhoods = neighbourhoods.loc[:,['Postcode', 'Borough', 'Neighbourhood']]

In [10]:
neighbourhoods.shape

(212, 3)

### Add longtitude and latitude

In [21]:
geo_coordinates_url = 'http://cocl.us/Geospatial_data'

geo_coordinates = pd.read_csv(geo_coordinates_url)

geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
neighbourhoods_with_lng_lat = pd.merge(neighbourhoods, geo_coordinates, left_on="Postcode", right_on='Postal Code')

neighbourhoods_with_lng_lat.drop(['Postal Code'], axis=1)

neighbourhoods_with_lng_lat.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
3,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
4,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
